# Imports

In [1]:
import pandas as pd
import pyarrow as pa
import numpy as np

from datetime import datetime, timedelta

In [2]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

# Read Data

In [41]:
symbol = "BTCUSDT"

df = pd.read_parquet(f"../data_binance_crypto/symbol={symbol}/", engine="pyarrow")
df.shape

(349290, 13)

In [42]:
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_vol,num_trades,taker_buy_base_asset_vol,taker_buy_quote_asset_vol,ignore_this,day
0,1609459200000,28923.63000000,28961.66000000,28913.12000000,28961.66000000,27.45703200,1609459259999,794382.04398665,1292,16.77719500,485390.82682460,0,2021-01-01
1,1609459260000,28961.67000000,29017.50000000,28961.01000000,29009.91000000,58.47750100,1609459319999,1695802.89696884,1651,33.73381800,978176.46820208,0,2021-01-01
2,1609459320000,29009.54000000,29016.71000000,28973.58000000,28989.30000000,42.47032900,1609459379999,1231358.69059884,986,13.24744400,384076.85445305,0,2021-01-01
3,1609459380000,28989.68000000,28999.85000000,28972.33000000,28982.69000000,30.36067700,1609459439999,880016.76348383,959,9.45602800,274083.07514154,0,2021-01-01
4,1609459440000,28982.67000000,28995.93000000,28971.80000000,28975.65000000,24.12433900,1609459499999,699226.20560386,726,6.81464400,197519.37488805,0,2021-01-01


## Quick and simple EDA

In [43]:
df.isna().sum()

open_time                    0
open                         0
high                         0
low                          0
close                        0
volume                       0
close_time                   0
quote_asset_vol              0
num_trades                   0
taker_buy_base_asset_vol     0
taker_buy_quote_asset_vol    0
ignore_this                  0
day                          0
dtype: int64

In [44]:
columns = df.columns
for col in sorted(columns):
    print(f"Col: {col.ljust(40, ' '):40} Type: {df[col].dtype}")

Col: close                                    Type: object
Col: close_time                               Type: int64
Col: day                                      Type: category
Col: high                                     Type: object
Col: ignore_this                              Type: object
Col: low                                      Type: object
Col: num_trades                               Type: int64
Col: open                                     Type: object
Col: open_time                                Type: int64
Col: quote_asset_vol                          Type: object
Col: taker_buy_base_asset_vol                 Type: object
Col: taker_buy_quote_asset_vol                Type: object
Col: volume                                   Type: object


In [45]:
float_columns = [
    "open",
    "high",
    "low",
    "close",
    "volume",
    "quote_asset_vol",
    "taker_buy_base_asset_vol",
    "taker_buy_quote_asset_vol",
    "ignore_this",
]

for col in float_columns:
    df[col] = df[col].astype(float)

In [46]:
df.describe()

,open_time,open,high,low,close,volume,close_time,quote_asset_vol,num_trades,taker_buy_base_asset_vol,taker_buy_quote_asset_vol,ignore_this
count,3.492900e+05,349290.000000,349290.000000,349290.000000,349290.000000,349290.000000,3.492900e+05,3.492900e+05,349290.000000,349290.000000,3.492900e+05,349290.0
mean,1.619954e+12,44291.498376,44329.903940,44253.400252,44291.562309,57.039721,1.619954e+12,2.450697e+06,1420.229265,28.079218,1.206923e+06,0.0
std,6.061868e+09,9600.903307,9602.704045,9598.646857,9600.898419,70.018457,6.061868e+09,2.843840e+06,1203.971846,36.765979,1.508756e+06,0.0
min,1.609459e+12,28241.950000,28764.230000,28130.000000,28235.470000,0.000000,1.609459e+12,0.000000e+00,0.000000,0.000000,0.000000e+00,0.0
25%,1.614700e+12,35398.997500,35436.632500,35360.000000,35399.000000,23.102220,1.614700e+12,1.012094e+06,776.000000,10.716988,4.689843e+05,0.0
50%,1.619967e+12,43857.955000,43915.200000,43800.025000,43858.660000,37.500648,1.619967e+12,1.668570e+06,1136.000000,17.937238,7.979389e+05,0.0
75%,1.625202e+12,53726.977500,53778.482500,53676.937500,53726.970000,64.550522,1.625203e+12,2.831762e+06,1677.000000,31.669046,1.389989e+06,0.0
max,1.630454e+12,64800.000000,64854.000000,64685.170000,64800.000000,2636.713888,1.630454e+12,1.136863e+08,42282.000000,2014.965612,8.947551e+07,0.0


# Feature engineering

In [47]:
df.head()

,open_time,open,high,low,close,volume,close_time,quote_asset_vol,num_trades,taker_buy_base_asset_vol,taker_buy_quote_asset_vol,ignore_this,day
0,1609459200000,28923.63,28961.66,28913.12,28961.66,27.457032,1609459259999,7.943820e+05,1292,16.777195,485390.826825,0.0,2021-01-01
1,1609459260000,28961.67,29017.50,28961.01,29009.91,58.477501,1609459319999,1.695803e+06,1651,33.733818,978176.468202,0.0,2021-01-01
2,1609459320000,29009.54,29016.71,28973.58,28989.30,42.470329,1609459379999,1.231359e+06,986,13.247444,384076.854453,0.0,2021-01-01
3,1609459380000,28989.68,28999.85,28972.33,28982.69,30.360677,1609459439999,8.800168e+05,959,9.456028,274083.075142,0.0,2021-01-01
4,1609459440000,28982.67,28995.93,28971.80,28975.65,24.124339,1609459499999,6.992262e+05,726,6.814644,197519.374888,0.0,2021-01-01


## Drop some columns

In [48]:
cols_to_drop = [
    "close_time",
    "quote_asset_vol",
    "taker_buy_base_asset_vol",
    "taker_buy_quote_asset_vol",
    "ignore_this"
]

df.drop(cols_to_drop, axis=1, inplace=True)

In [49]:
df.head()

,open_time,open,high,low,close,volume,num_trades,day
0,1609459200000,28923.63,28961.66,28913.12,28961.66,27.457032,1292,2021-01-01
1,1609459260000,28961.67,29017.50,28961.01,29009.91,58.477501,1651,2021-01-01
2,1609459320000,29009.54,29016.71,28973.58,28989.30,42.470329,986,2021-01-01
3,1609459380000,28989.68,28999.85,28972.33,28982.69,30.360677,959,2021-01-01
4,1609459440000,28982.67,28995.93,28971.80,28975.65,24.124339,726,2021-01-01


## Datetime semantics

In [50]:
df["quarter"] = pd.to_datetime(df["day"]).dt.quarter

In [51]:
df["month_of_year"] = pd.to_datetime(df["day"]).dt.month

In [52]:
df["week_of_year"] = pd.to_datetime(df["day"]).dt.isocalendar().week

In [53]:
df["day_of_month"] = pd.to_datetime(df["day"]).dt.day

In [54]:
df["day_of_week"] = pd.to_datetime(df["day"]).dt.dayofweek

In [63]:
df["hour_of_day"] = pd.to_datetime(df["open_time"], unit="ms").dt.hour

In [65]:
df["minute_of_hour"] = pd.to_datetime(df["open_time"], unit="ms").dt.minute

In [66]:
df.head()

,open_time,open,high,low,close,volume,num_trades,day,quarter,month_of_year,week_of_year,day_of_month,day_of_week,hour_of_day,minute_of_hour
0,1609459200000,28923.63,28961.66,28913.12,28961.66,27.457032,1292,2021-01-01,1,1,53,1,4,0,0
1,1609459260000,28961.67,29017.50,28961.01,29009.91,58.477501,1651,2021-01-01,1,1,53,1,4,0,1
2,1609459320000,29009.54,29016.71,28973.58,28989.30,42.470329,986,2021-01-01,1,1,53,1,4,0,2
3,1609459380000,28989.68,28999.85,28972.33,28982.69,30.360677,959,2021-01-01,1,1,53,1,4,0,3
4,1609459440000,28982.67,28995.93,28971.80,28975.65,24.124339,726,2021-01-01,1,1,53,1,4,0,4


In [67]:
df["minute_of_hour"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59])

# Place holder